# Approximating the flow map $\Phi_{\mathcal{F}}^{0.05}$ of the SIR model

The equations of motion we consider correspond to those of the SIR model with a specific choice of its parameters. The dynamics can be described as 
$$
\dot{\mathbf{x}}(t) = \begin{bmatrix} -x_1(t)x_2(t)\\ x_1(t)x_2(t)-x_2(t) \\x_2(t) \end{bmatrix} =: \mathcal{F}(\mathbf{x}(t)).
$$
These equations have the property that
$$
\frac{d}{dt}\left(x_1(t)+x_2(t)+x_3(t)\right) =0
$$
and hence the linear energy function $\mathcal{E}(\mathbf{x}) = 1^T\mathbf{x}=x_1+x_2+x_3$ is conserved by the solutions of the differential equation.

We train two neural networks that aim to approximate $\Phi^{0.05}_{\mathcal{F}}$ for initial conditions in the unit box $[0,1]^3\subset\mathbb{R}^3$. We do so with a network constrained to replicate the conservation of $\mathcal{E}$, and one which is left free not to replicate such conservation law. 

In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from generateData import *
import time

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
torch.set_default_tensor_type(torch.DoubleTensor)

## Initialization of DATA

In [4]:
n = 3 #Input dimension
o = 3 #Output dimension

In [5]:
N = 5000
dim = n
Ntrain = 0.9
noisy = False
M = 6
T = 0.25
x_train, y_train, x_test, y_test, h = generate(N,M,T,Ntrain,noisy)

In [6]:
from torch.utils.data import Dataset, DataLoader
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.from_numpy(x.astype(np.float64))
    self.y = torch.from_numpy(y.astype(np.float64))
    self.length = self.x.shape[0]

  def __getitem__(self,idx):
    return self.x[idx], self.y[idx]
  def __len__(self):
    return self.length #num elementi del dataset

batch_size = 32

trainset = dataset(x_train,y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = dataset(x_test,y_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

### Structure preserving architecture

In [7]:
import torch.nn.functional as F

class DynamicBlock(nn.Module):
    def __init__(self, nlayer, input, lift):
        super(DynamicBlock, self).__init__()
        
        self.nlayers = nlayer
        self.input = input
        self.lift = lift
        self.DOF = int(self.input * (self.input-1) / 2)

        self.A = nn.ModuleList([nn.Linear(self.input,self.lift,bias=False) for i in range(self.nlayers)])
        self.B = nn.ModuleList([nn.Linear(self.lift,self.DOF) for i in range(self.nlayers)])
        self.Biases = nn.ParameterList([nn.Parameter(torch.randn(self.lift)) for i in range(self.nlayers)])
        self.nl = nn.ReLU()

        self.dts = nn.Parameter(torch.rand(self.nlayers))

        self.alpha = torch.ones(self.input)

    def buildSkew(self,ff):
        res = torch.zeros((len(ff),self.input,self.input))
        iu1 = torch.triu_indices(self.input,self.input,1)
        res[:,iu1[0],iu1[1]] = ff
        res = res - torch.transpose(res,1,2)
        return res

    def forward(self, x):
        
        for i in range(self.nlayers):
            c = self.A[i](x)+self.Biases[i]
            ff = self.nl(self.B[i](self.nl(self.A[i](x)+self.Biases[i])))
            Mat = self.buildSkew(ff)
            x = x + 0.1 * self.alpha @ Mat
        return x

#Linear lifting layer that preserves the sum
class Lift(nn.Module):
    def __init__(self, lift):
        super(Lift, self).__init__()
        self.outputDim = lift
    def forward(self,x):
        I = torch.eye(x.shape[1])
        Z = torch.zeros((x.shape[1],self.outputDim-x.shape[1]))
        liftMat = torch.cat((I,Z),dim=1)
        return x@liftMat

#Projection layer that preserves the sum
class Projection(nn.Module):
    def __init__(self, inputDim, lowerDim):
        super(Projection, self).__init__()
        self.outputDim = lowerDim
        self.input = inputDim
    def forward(self,x):
        I = torch.eye(self.outputDim)
        Z = torch.zeros((self.input-self.outputDim,self.outputDim))
        projMat = torch.cat((I,Z),dim=0)
        s = torch.sum(x[:,self.outputDim:],dim=1).unsqueeze(1)/(self.outputDim)
        y = x @ projMat
        y = y + s
        return y        

class Network(nn.Module):
    def __init__(self, input, output):
        super(Network, self).__init__()
        
        self.input = input
        self.output = output
        dim = [3,10,10,15]
        lift = max(dim)
        lista = []
        for i in np.arange(1,len(dim)):
            if dim[i]>dim[i-1]:
                lista.append(Lift(dim[i]))
            else:
                lista.append(Projection(dim[i-1],dim[i]))
                
            #lista.append(DynamicBlock(3,dim[i],max(dim)))
            lista.append(DynamicBlock(3,dim[i],50))

        lista.append(Projection(dim[-1],o))
        self.seq = nn.Sequential(*lista)

    def forward(self, x):
        x = self.seq(x)
        return x

net = Network(n, o)

### Unstructured architecture

In [8]:
class UnstructuredNetwork(nn.Module):
    def __init__(self, input, output, nlayers):
        super(UnstructuredNetwork, self).__init__()
        
        self.input = input
        self.output = output
        
        self.nlayers = nlayers
        
        self.lift = nn.Linear(input,10)
        self.linears = nn.ModuleList([nn.Linear(10,10) for _ in range(self.nlayers)])
        self.proj = nn.Linear(10,output)
        
        self.dts = nn.Parameter(torch.rand(self.nlayers))

        self.nl = nn.Tanh()

    def forward(self, x):
        x = self.lift(x)
        for i in range(self.nlayers):
            x = x + 0.1 * self.nl(self.linears[i](x))
        return self.proj(x)
    
netU = UnstructuredNetwork(n,o,5)

### Network training

In [9]:
def train_network(model,epochs,optimizer,criterion,scheduler):
  num_epochs = int(0.1 * epochs)

  for epoch in tqdm(range(epochs)):
      running_loss = 0.0
      for i, data in enumerate(trainloader, 0):
          
          inputs, outputs = data[0], data[1]
          optimizer.zero_grad()
          loss = 0
          for k in range(M-1):
            inputs = model(inputs)
            loss += criterion(inputs,outputs[:,:,k])

          loss.backward()
          optimizer.step()
          
      if epoch % num_epochs == 0:  
          print('[Epoch %d] loss: %.10f' %
                (epoch + 1,loss.item()))

      scheduler.step()
      
  print('Finished Training')

In [10]:
import torch.optim as optim

criterion = nn.MSELoss()
lr = 1e-2
epochs = 300

optimizer = torch.optim.Adam(net.parameters(),lr=lr)
optimizerU = torch.optim.Adam(netU.parameters(),lr=lr)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=int(0.45*epochs), gamma=0.1)
schedulerU = torch.optim.lr_scheduler.StepLR(optimizerU, step_size=int(0.45*epochs), gamma=0.1)

In [11]:
train = False

if train:
    print("=====================================")
    print("Training of the unconstrained network")
    print("=====================================")
    train_network(net,epochs,optimizer,criterion,scheduler)
    print("=====================================")
    print("Training of the constrained network")
    print("=====================================")
    train_network(netU,epochs,optimizerU,criterion,schedulerU)
    timestamp = time.strftime("%Y%m%d_%H%M%S") 

    torch.save(net.state_dict(), f"constrainedNet_{timestamp}.pt")
    torch.save(netU.state_dict(), f"unconstrainedNet_{timestamp}.pt")

else:
    net.load_state_dict(torch.load("constrainedNet.pt",map_location='cpu'))
    netU.load_state_dict(torch.load("unconstrainedNet.pt",map_location='cpu'))

net.eval();
netU.eval();

### Evaluation

In [12]:
data = next(iter(testloader))
input, output = data[0], data[1]
predSol = torch.zeros((len(input),dim,M-1))
predSolU = torch.zeros((len(input),dim,M-1))

predSol[:,:,0] = net(input)
predSolU[:,:,0] = netU(input)

for i in range(1,M-1):
    predSol[:,:,i] = net(predSol[:,:,i-1])
    predSolU[:,:,i] = netU(predSolU[:,:,i-1])
    
print("Test error: ",criterion(predSol,output))
print("Test error U : ",criterion(predSolU,output))
print("Sum first output: ",torch.sum(net(input[0:1])).item())
print("Sum first output U: ",torch.sum(netU(input[0:1])).item())
print("Sum first input: ",torch.sum(input[0:1]).item())

Test error:  tensor(3.1461e-08, grad_fn=<MseLossBackward0>)
Test error U :  tensor(4.2725e-08, grad_fn=<MseLossBackward0>)
Sum first output:  1.2964816152453218
Sum first output U:  1.2964517757097522
Sum first input:  1.296481615245322


In [13]:
def flowC(y):
    z = torch.from_numpy(y.astype(np.float64)).reshape(1,-1)
    return net(z).detach().numpy().reshape(-1)

def flowU(y):
    z = torch.from_numpy(y.astype(np.float64)).reshape(1,-1)
    return netU(z).detach().numpy().reshape(-1)

y0 = np.array([0.3,0.2,0.1]) #np.random.rand(dim)
y0 = y0/np.sum(y0)
R0 = 1
f = lambda t,y: np.array([-R0*y[0]*y[1],
                            R0*y[0]*y[1] - y[1],
                            y[1]
    ])

point = y0.copy()
pointU = y0.copy()
tt = 0
it = 100
mat = np.zeros((3,it))
mat[:,0] = point
matU = np.zeros((3,it))
matU[:,0] = point
time = [0]
for c in range(it-1):
    tt+=h
    time.append(tt)
    point = flowC(point)
    pointU = flowU(pointU)
    mat[:,c+1] = point
    matU[:,c+1] = pointU
sol_true = scipy.integrate.solve_ivp(f,[0,max(time)],y0,method='RK45',t_eval=time,atol=1e-11,rtol=1e-11).y

In [16]:
import matplotlib

#Setting the plotting parameters
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'
matplotlib.rcParams['font.size']= 45
matplotlib.rcParams['font.family']= 'ptm' #'Times New Roman


fig = plt.figure(figsize=(20,10),dpi=600)

plt.plot(time,sol_true[0,:],'-',c="blue",label=r"$x$ reference",linewidth=6)
plt.plot(time,sol_true[1,:],'-',c="green",label=r"$y$ reference",linewidth=6)
plt.plot(time,sol_true[2,:],'-',c="purple",label=r"$z$ reference",linewidth=6)

plt.plot(time,mat[0,:],'-o',c="cyan",label=r"$x$ constrained",linewidth=3,markersize=8)#,markevery=3)#,alpha=0.7)
plt.plot(time,mat[1,:],'-o',c="gold",label=r"$y$ constrained",linewidth=3,markersize=8)#,markevery=3)#,alpha=0.7)
plt.plot(time,mat[2,:],'-o',c="chocolate",label=r"$z$ constrained",linewidth=3,markersize=8)#,markevery=3)#,alpha=0.7)

plt.plot(time,matU[0,:],'-o',c="magenta",label=r"$x$ unconstrained",linewidth=1,markersize=4)#,markevery=3)
plt.plot(time,matU[1,:],'-o',c="black",label=r"$y$ unconstrained",linewidth=1,markersize=4)#,markevery=3)
plt.plot(time,matU[2,:],'-o',c="lime",label=r"$z$ unconstrained",linewidth=1,markersize=4)#,markevery=3)

plt.legend(fontsize=25,loc='upper center', bbox_to_anchor=(0.5, 1.25), ncol=3, fancybox=True, shadow=True)

#plt.title("Behaviour SIR model",fontsize=30)
plt.tick_params(axis='both', which='major', labelsize=30)
plt.xlabel(r"$t$")

plt.savefig("SIRModel.pdf",bbox_inches='tight')

plt.show();

fig = plt.figure(figsize=(20,10),dpi=600)
plt.semilogy(time,np.abs(1-np.sum(mat,axis=0)),'ko',label=r"$|\delta \mathcal{E}(t)|$ constrained",markersize=10)
plt.semilogy(time,np.abs(1-np.sum(matU,axis=0)),'rd',label=r"$|\delta \mathcal{E}(t)|$ unconstrained",markersize=10)
plt.legend(fontsize=25,loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=4, fancybox=True, shadow=True)

plt.xlabel(r"$t$")
plt.ylabel(r"$|\delta\mathcal{E}(t)|$")
plt.savefig("EnergyError.pdf",bbox_inches='tight')
plt.show();
